In [1]:
from scipy.optimize import rosen, rosen_der
from cyipopt import minimize_ipopt
import time

In [22]:
from jax.config import config

# Enable 64 bit floating point precision
config.update("jax_enable_x64", True)

# We use the CPU instead of GPU und mute all warnings if no GPU/TPU is found.
config.update('jax_platform_name', 'cpu')

import jax.numpy as np
from jax import jit, grad, jacfwd, jacrev
from cyipopt import minimize_ipopt

In [23]:
def objective(x):
    return x[0]*x[3]*np.sum(x[:3]) + x[2]

def eq_constraints(x):
    return np.sum(x**2) - 40

def ineq_constrains(x):
    return np.prod(x) - 25

In [24]:
# jit the functions
obj_jit = jit(objective)
con_eq_jit = jit(eq_constraints)
con_ineq_jit = jit(ineq_constrains)

# build the derivatives and jit them
obj_grad = jit(grad(obj_jit))  # objective gradient
obj_hess = jit(jacrev(jacfwd(obj_jit))) # objective hessian
con_eq_jac = jit(jacfwd(con_eq_jit))  # jacobian
con_ineq_jac = jit(jacfwd(con_ineq_jit))  # jacobian
con_eq_hess = jacrev(jacfwd(con_eq_jit)) # hessian
con_eq_hessvp = jit(lambda x, v: con_eq_hess(x) * v[0]) # hessian vector-product
con_ineq_hess = jacrev(jacfwd(con_ineq_jit))  # hessian
con_ineq_hessvp = jit(lambda x, v: con_ineq_hess(x) * v[0]) # hessian vector-product

In [28]:
# constraints
cons = [
    {'type': 'eq', 'fun': con_eq_jit, 'jac': con_eq_jac, 'hess': con_eq_hessvp},
    {'type': 'ineq', 'fun': con_ineq_jit, 'jac': con_ineq_jac, 'hess': con_ineq_hessvp}
 ]

# starting point
x0 = np.array([1.0, 5.0, 5.0, 1.0])

# variable bounds: 1 <= x[i] <= 5
bnds = [(1, 5) for _ in range(x0.size)]

t0 = time.time()
# executing the solver
res = minimize_ipopt(obj_jit, jac=obj_grad, hess=obj_hess, x0=x0, bounds=bnds,
                  constraints=cons, options={'disp': 5})
print(f"fist_method time : { time.time()-t0 }")
print("hello")

This is Ipopt version 3.14.11, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:       10

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        4
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.6109693e+01 1.12e+01 5.28e-01   0.0 0.00e+00    -  0.00e+00 0.00e+00   0
   

In [60]:
x_test = np.asarray(x0)
con_ineq_jac(x_test)
con_eq_jac(x_test)
# con_eq_jac(np.asarray(x0))
np.append(con_eq_jac(x_test), con_ineq_jac(x_test))

Array([ 2., 10., 10.,  2., 25.,  5.,  5., 25.], dtype=float64)

In [98]:
import cyipopt
import jax.numpy as np
class HS071():

    def objective(self, x):
        """Returns the scalar value of the objective given x."""
#         return x[0] * x[3] * np.sum(x[0:3]) + x[2]
        return obj_jit(x)

    def gradient(self, x):
        """Returns the gradient of the objective with respect to x."""
        return obj_grad(x)

    def constraints(self, x):
        """Returns the constraints."""
#         return np.array((np.prod(x), np.dot(x, x)))
        return np.append( con_ineq_jit(x), con_eq_jit(x) )

    def jacobian(self, x):
        """Returns the Jacobian of the constraints with respect to x."""
#         return np.concatenate((np.prod(x)/x, 2*x))
        return np.append(con_ineq_jac(x), con_eq_jac(x))

    def hessianstructure(self):
        """Returns the row and column indices for non-zero vales of the
        Hessian."""

        # NOTE: The default hessian structure is of a lower triangular matrix,
        # therefore this function is redundant. It is included as an example
        # for structure callback.

        return np.nonzero(np.tril(np.ones((4, 4))))

#     def hessian(self, x, lagrange, obj_factor):
#         """Returns the non-zero values of the Hessian."""

#         H = obj_factor*np.array((
#             (2*x[3], 0, 0, 0),
#             (x[3],   0, 0, 0),
#             (x[3],   0, 0, 0),
#             (2*x[0]+x[1]+x[2], x[0], x[0], 0)))

#         H += lagrange[0]*np.array((
#             (0, 0, 0, 0),
#             (x[2]*x[3], 0, 0, 0),
#             (x[1]*x[3], x[0]*x[3], 0, 0),
#             (x[1]*x[2], x[0]*x[2], x[0]*x[1], 0)))

#         H += lagrange[1]*2*np.eye(4)

#         row, col = self.hessianstructure()

#         return H[row, col]

    def intermediate(self, alg_mod, iter_count, obj_value, inf_pr, inf_du, mu,
                     d_norm, regularization_size, alpha_du, alpha_pr,
                     ls_trials):
        """Prints information at every Ipopt iteration."""

        msg = "Objective value at iteration #{:d} is - {:g}"

        print(msg.format(iter_count, obj_value))
lb = [1.0, 1.0, 1.0, 1.0]
ub = [5.0, 5.0, 5.0, 5.0]

cl = [0.0, 0.0]
cu = [2.0e19, 0.0]
x0 = [1.0, 5.0, 5.0, 1.0]
nlp = cyipopt.Problem(
   n=len(x0),
   m=len(cl),
   problem_obj=HS071(),
   lb=lb,
   ub=ub,
   cl=cl,
   cu=cu,
)
nlp.add_option('mu_strategy', 'adaptive')
nlp.add_option('tol', 1e-7)
x, info = nlp.solve(x0)

Objective value at iteration #0 is - 16.1097
This is Ipopt version 3.14.11, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        4
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.6109693e+01 1.12e+01 5.28e-01  

In [99]:
print(f"solution: {x}")
obj_jit(x)
con_eq_jit(x)

solution: [0.99999999 4.74299964 3.82114998 1.37940831]


Array(7.10542736e-15, dtype=float64)

In [100]:
import cyipopt
import numpy as np
class HS071_full():

    def objective(self, x):
        """Returns the scalar value of the objective given x."""
#         return x[0] * x[3] * np.sum(x[0:3]) + x[2]
        return obj_jit(x)

    def gradient(self, x):
        """Returns the gradient of the objective with respect to x."""
        return np.array([
            x[0]*x[3] + x[3]*np.sum(x[0:3]),
            x[0]*x[3],
            x[0]*x[3] + 1.0,
            x[0]*np.sum(x[0:3])
        ])

    def constraints(self, x):
        """Returns the constraints."""
        return np.array((np.prod(x), np.dot(x, x)))

    def jacobian(self, x):
        """Returns the Jacobian of the constraints with respect to x."""
        return np.concatenate((np.prod(x)/x, 2*x))

    def hessianstructure(self):
        """Returns the row and column indices for non-zero vales of the
        Hessian."""

        # NOTE: The default hessian structure is of a lower triangular matrix,
        # therefore this function is redundant. It is included as an example
        # for structure callback.

        return np.nonzero(np.tril(np.ones((4, 4))))

    def hessian(self, x, lagrange, obj_factor):
        """Returns the non-zero values of the Hessian."""

        H = obj_factor*np.array((
            (2*x[3], 0, 0, 0),
            (x[3],   0, 0, 0),
            (x[3],   0, 0, 0),
            (2*x[0]+x[1]+x[2], x[0], x[0], 0)))

        H += lagrange[0]*np.array((
            (0, 0, 0, 0),
            (x[2]*x[3], 0, 0, 0),
            (x[1]*x[3], x[0]*x[3], 0, 0),
            (x[1]*x[2], x[0]*x[2], x[0]*x[1], 0)))

        H += lagrange[1]*2*np.eye(4)

        row, col = self.hessianstructure()

        return H[row, col]

    def intermediate(self, alg_mod, iter_count, obj_value, inf_pr, inf_du, mu,
                     d_norm, regularization_size, alpha_du, alpha_pr,
                     ls_trials):
        """Prints information at every Ipopt iteration."""

        msg = "Objective value at iteration #{:d} is - {:g}"

        print(msg.format(iter_count, obj_value))
lb = [1.0, 1.0, 1.0, 1.0]
ub = [5.0, 5.0, 5.0, 5.0]

cl = [25.0, 40.0]
cu = [2.0e19, 40.0]
x0 = [1.0, 5.0, 5.0, 1.0]
nlp = cyipopt.Problem(
   n=len(x0),
   m=len(cl),
   problem_obj=HS071_full(),
   lb=lb,
   ub=ub,
   cl=cl,
   cu=cu,
)
nlp.add_option('mu_strategy', 'adaptive')
nlp.add_option('tol', 1e-7)
x_full, info = nlp.solve(x0)

Objective value at iteration #0 is - 16.1097
Objective value at iteration #1 is - 17.4104
This is Ipopt version 3.14.11, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:       10

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        4
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_p

In [101]:
print(f"solution: {x_full}")
obj_jit(x_full)
con_ineq_jit(x_full)

solution: [0.99999999 4.74299964 3.82114998 1.37940829]


Array(-2.49981998e-07, dtype=float64)

In [80]:
obj_jit(np.asarray(x0))

Array(16., dtype=float64)

In [1]:
import jax
import jax.numpy as jnp
from jax import grad, jit, jacrev

In [2]:
goal = jnp.array([3.0,3.0])
def func(x):
    sum = 0
    for i in range(2):
        sum += jnp.sum( jnp.square(x-goal) )
    return sum
func_grad = grad(func, 0)

def func2(x):
    return x-goal
func2_grad = jacrev(func2, 0)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [3]:
x = jnp.array([1.0, 1.0])
func(x)

Array(16., dtype=float32)

In [4]:
func_grad(x)

Array([-8., -8.], dtype=float32)

In [5]:
func2(x)

Array([-2., -2.], dtype=float32)

In [6]:
func2_grad(x)

Array([[1., 0.],
       [0., 1.]], dtype=float32)

In [17]:
a = jnp.array([[1,3],[2,4]])

In [18]:
a

Array([[1, 3],
       [2, 4]], dtype=int32)

In [21]:
b = a.T.reshape(-1,1)

In [22]:
b

Array([[1],
       [2],
       [3],
       [4]], dtype=int32)

In [14]:
b.reshape(2,2)

Array([[1, 3],
       [2, 4]], dtype=int32)

In [24]:
c = jnp.array([1,2,3,4])

In [28]:
c.reshape(2,2, order='F')

Array([[1, 3],
       [2, 4]], dtype=int32)